In [ ]:
import openai
import os
import csv
import librosa
import numpy as np
from pydub import AudioSegment

def transcribe_audio(api_key, audio_file):
    client = openai.OpenAI(api_key=api_key)

    with open(audio_file, "rb") as file:
        response = client.audio.transcriptions.create(
            model="whisper-1",
            file=file
        )

    return response.text  # Extract the transcribed text

def extract_impulsiveness(audio_file):
    """Calculates pitch and amplitude variations to measure impulsiveness."""
    y, sr = librosa.load(audio_file, sr=None)

    # Extract pitch (fundamental frequency)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_values = [np.max(pitches[:, i]) for i in range(pitches.shape[1]) if np.max(pitches[:, i]) > 0]

    # Extract amplitude variations using RMS energy
    rms_energy = librosa.feature.rms(y=y)[0]

    # Calculate sudden changes in pitch
    pitch_changes = np.abs(np.diff(pitch_values))  # Absolute change in pitch
    amp_changes = np.abs(np.diff(rms_energy))  # Absolute change in loudness

    # Define an impulsiveness score (higher = more impulsive)
    impulsiveness_score = (np.mean(pitch_changes) + np.mean(amp_changes)) * 100

    return impulsiveness_score

def save_to_csv(results, output_file="/content/output_folder/output.csv"):
    """Save impulsiveness scores to a CSV file."""
    os.makedirs("/content/output_folder", exist_ok=True)

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["File Name", "Impulsiveness Score"])
        writer.writerows(results)

def process_multiple_files(api_key, audio_files, output_file):
    results = []

    for audio_file in audio_files:
        print(f"Processing {audio_file}...")
        transcribe_audio(api_key, audio_file)  # Using Whisper API
        impulsiveness_score = extract_impulsiveness(audio_file)
        results.append([os.path.basename(audio_file), impulsiveness_score])

    save_to_csv(results, output_file)
    print(f"All data saved to {output_file}")

if __name__ == "__main__":
    API_KEY = ""  # Replace with your OpenAI API key
    AUDIO_FILES = [
        "/content/ABB_segment_2 (1).mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3",
        "/content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3"
    ]  # Replace with your list of audio files
    OUTPUT_FILE = "/content/output_folder/output_impulsiveness.csv"

    process_multiple_files(API_KEY, AUDIO_FILES, OUTPUT_FILE)


Processing /content/ABB_segment_2 (1).mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jan-31-2024 - Audio_segment_2.mp3...
Processing /content/Adani Wilmar Limited (NSEI_AWL) Jul-30-2024 - Audio_segment_2.mp3...
All data saved to /content/output_folder/output_impulsiveness.csv


In [2]:
!pip install pydub
